In [27]:
import numpy as np

In [1]:
class link:
    def __init__(self, name, a0, a1):
        self.name = name
        self.travel_time = 0 # the travel time of the link
        self._flow = 0 # the flow on the link
        self._paths = [] # the paths using this link
        self._a0 = a0
        self._a1 = a1
        
    def add_path(self, path):
        self._paths.append(path)
    
    def update_flow(self):
        self._flow = 0
        for path in self._paths:
            self._flow += path.flow
    
    def update_travel_time(self):
        self.update_flow()
        self.travel_time = self._a0 + self._a1 * self._flow
        
    def __str__(self):
        return self.name

In [18]:
class path:
    def __init__(self, name, links):
        self.name = name
        self._travel_time = 0 # the travel time of the paths
        self.flow = 0 # the flow on the paths
        self._links = links # the paths using this link
        for link in links:
            link.add_path(self)
    
    def get_travel_time(self):
        return self._travel_time
    
    def update_travel_time(self):
        self._travel_time = 0
        for link in self._links:
            self._travel_time += link.travel_time
            
    def __str__(self):
        return self.name

In [96]:
class network:
    def __init__(self, paths, links):
        self.paths = paths
        self.links = links
    
    def update(self):
        for link in self.links:
            link.update_travel_time()
        for path in self.paths:
            path.update_travel_time()
            
    def update_av(self, avs):
        for path in self.paths:
            path.flow = 0
        for av in avs:
            av.path.flow += av.flow
        self.update()
        for av in avs:
            av.give_reward()

In [97]:
ab = link('ab', 1, 1/100)
ac = link('ac', 2, 0)
bc = link('bc', 0.25, 0)
bd = link('bd', 2, 0)
cd = link('cd', 1, 1/100)

In [98]:
abcd = path('abcd', [ab, bc, cd])
abd = path('abd', [ab, bd])
acd = path('acd', [ac, cd])

In [99]:
braess = network([abd, acd, abcd], [ab, ac, bc, bd, cd])

In [100]:
braess.update()

In [101]:
#Nash equilibrium
abcd.flow = 50
abd.flow = 25
acd.flow = 25
braess.update()
print([path.get_travel_time() for path in braess.paths])

[3.75, 3.75, 3.75]


In [102]:
#social optimum
abcd.flow = 0
abd.flow = 50
acd.flow = 50
braess.update()
print([path.get_travel_time() for path in braess.paths])

[3.5, 3.5, 3.25]


In [103]:
class autonomous_vehicle:
    p = 0.5
    def __init__(self, name, flow):
        self.name = name
        self.path = None
        self.reward = 0
        self.flow = flow
    
    def path_choice(self, path):
        if self.path == None:
            self.path = path
        self.path = (path if np.random.rand() < self.p else self.path)
    
    def give_reward(self):
        self.reward = - self.path.get_travel_time()

In [104]:
av1 = autonomous_vehicle('1', 25)
av2 = autonomous_vehicle('2', 25)
av3 = autonomous_vehicle('3', 25)
av4 = autonomous_vehicle('4', 25)

avs = [av1, av2, av3, av4]

In [144]:
best_path = [x for _,x in sorted([(path.get_travel_time(),path) for path in braess.paths],  key=lambda tup: tup[0])][0]
for av in avs:
    av.path_choice(best_path)

braess.update_av(avs)

In [143]:
print([av.reward for av in avs])
print([path.get_travel_time() for path in braess.paths])

[-3.75, -3.75, -3.25, -3.75]
[3.75, 3.25, 3.25]


In [120]:
[x for _,x in sorted([(path.get_travel_time(),path) for path in braess.paths])][0]

TypeError: unorderable types: path() < path()

In [122]:
print(sorted([(path.get_travel_time(),path) for path in braess.paths]))

TypeError: unorderable types: path() < path()

In [131]:
print(sorted([(path.get_travel_time(), path) for path in braess.paths],  key=lambda tup: tup[0]))

[(3.25, <__main__.path object at 0x11193a4e0>), (3.25, <__main__.path object at 0x11193a358>), (3.75, <__main__.path object at 0x11193a320>)]
